# Metrics calculation (LLM)

In [1]:
import pandas as pd
# summary_dataset = pd.read_csv("summary_dataset_with_bertscore.csv")
# columns = [column for column in summary_dataset.columns if "bertscore" not in column] 
# print(columns)
# summary_dataset = summary_dataset[columns]
# summary_dataset.to_csv("summary_dataset.csv", index=False)
# summary_dataset = pd.read_csv("summary_dataset.csv")
# summary_dataset.head()

In [2]:
summary_dataset = pd.read_csv("summary_dataset_with_bertscore_rouge_meteor.csv")
summary_dataset = summary_dataset.rename(columns={"Unnamed: 0": "Ind"})
import random

random.seed(42)
indices_for_evaluation = random.sample(range(0, len(summary_dataset)), 100)
indices_for_evaluation = sorted(indices_for_evaluation)
summary_dataset = summary_dataset[summary_dataset["Ind"].isin(indices_for_evaluation)]

ValueError: Sample larger than population or is negative

In [3]:
summary_files = [
    "llama_7b_predictions.csv",
    "starling_predictions.csv",
    "yagpt_predictions.csv",
    "yagpt3_predictions.csv"
]

llm_data = pd.DataFrame()
for summary_file in summary_files:
    model_name = summary_file.split(".")[0]
    df = pd.read_csv(summary_file)[[model_name]]
    llm_data = pd.concat([llm_data, df], axis=1)

llm_data.head()

,llama_7b_predictions,starling_predictions,yagpt_predictions,yagpt3_predictions
0,"Bluetooth постоянно включен, это опасно, так к...",Постоянно включенный Bluetooth может создавать...,- Постоянно включенный Bluetooth на смартфоне ...,Доцент кафедры информатики РЭУ им. Плеханова А...
1,Китайское издание Sina высоко оценило новый ро...,Китайский издание Sina высоко оценило новый ро...,"К сожалению, я не могу ничего сказать об этом....","К сожалению, я не могу ничего сказать об этом...."
2,8 сентября в Японии началась избирательная кам...,"Всего три кандидата — Есихидэ Суга, Сигэру Иси...","К сожалению, я не могу ничего сказать об этом....","К сожалению, я не могу ничего сказать об этом...."
3,"Мария Колесникова, член президиума координацио...","Мария Колесникова, член президиума координацио...",- Член президиума координационного совета оппо...,"Мария Колесникова, член президиума координацио..."
4,Аналитики Deutsche Bank предсказали наступлени...,"Аналитики Deutsche Bank предполагают, что 2020...",- Аналитики Deutsche Bank предсказали наступле...,Аналитики Deutsche Bank предсказали наступлени...


In [4]:
summary_dataset = pd.concat([summary_dataset, llm_data.set_index(summary_dataset.index)], axis=1)
summary_dataset = summary_dataset[[column for column in summary_dataset.columns if "bertscore" not in column and "rouge" not in column and "meteor" not in column]]
summary_dataset = summary_dataset[summary_dataset["yagpt_predictions"] != "К сожалению, я не могу ничего сказать об этом. Давайте сменим тему?"]
summary_dataset.head()
print("Total: ", len(summary_dataset))

Total:  65


## BERTScore

In [5]:
from evaluate import load
bertscore = load("bertscore")
files = [
    "mbart_predictions.txt",
    "mt5_predictions.txt",
    "summarunner_predictions.txt",
    "llama_7b_predictions.csv",
    "starling_predictions.csv",
    "yagpt_predictions.csv",
    "yagpt3_predictions.csv"
]

In [6]:
results = {}
references = list(summary_dataset["summary"])
for filename in files:
    predictions_field = filename.split(".")[0]
    predictions = list(summary_dataset[predictions_field])
    results[predictions_field] = bertscore.compute(predictions=predictions, references=references, lang="ru", verbose=True, model_type="microsoft/mdeberta-v3-base", nthreads=16, batch_size=16, use_fast_tokenizer=False)

calculating scores...
computing bert embedding.


  0%|          | 0/9 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/5 [00:00<?, ?it/s]

done in 45244.20 seconds, 0.00 sentences/sec
calculating scores...
computing bert embedding.


...

done in 45245.90 seconds, 0.00 sentences/sec
calculating scores...
computing bert embedding.


  0%|          | 0/9 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/5 [00:00<?, ?it/s]

done in 45246.26 seconds, 0.00 sentences/sec


In [7]:
data_keys = list(results.keys())
for data_key in data_keys:
    old_keys = list(results[data_key].keys())
    for old_metric_key in old_keys:
        results[data_key][f"{data_key}_bertscore_" + old_metric_key] = results[data_key].pop(old_metric_key)

In [8]:
print(results[filename.split(".")[0]].keys())
print(len(results[filename.split(".")[0]]))

dict_keys(['yagpt3_predictions_bertscore_precision', 'yagpt3_predictions_bertscore_recall', 'yagpt3_predictions_bertscore_f1', 'yagpt3_predictions_bertscore_hashcode'])
4


In [9]:
print(len(results[filename.split(".")[0]][f"{filename.split('.')[0]}_bertscore_precision"]))

65


In [10]:
new_data = {}
for data_key in results.keys():
    for metric_key in results[data_key].keys():
        if "hashcode" in metric_key:
            continue
        new_data[metric_key] = pd.Series(results[data_key][metric_key])
new_data_df = pd.DataFrame(new_data)
new_data_df.head()

,mbart_predictions_bertscore_precision,mbart_predictions_bertscore_recall,mbart_predictions_bertscore_f1,mt5_predictions_bertscore_precision,mt5_predictions_bertscore_recall,mt5_predictions_bertscore_f1,summarunner_predictions_bertscore_precision,summarunner_predictions_bertscore_recall,summarunner_predictions_bertscore_f1,llama_7b_predictions_bertscore_precision,...,llama_7b_predictions_bertscore_f1,starling_predictions_bertscore_precision,starling_predictions_bertscore_recall,starling_predictions_bertscore_f1,yagpt_predictions_bertscore_precision,yagpt_predictions_bertscore_recall,yagpt_predictions_bertscore_f1,yagpt3_predictions_bertscore_precision,yagpt3_predictions_bertscore_recall,yagpt3_predictions_bertscore_f1
0,0.634706,0.662176,0.648150,0.697221,0.585219,0.636329,0.618635,0.642960,0.630563,0.626199,...,0.632462,0.671103,0.657498,0.664231,0.655310,0.665903,0.660564,0.605509,0.668242,0.635331
1,0.779796,0.720445,0.748946,0.819961,0.630633,0.712942,0.667503,0.665685,0.666593,0.662520,...,0.667891,0.693992,0.739694,0.716114,0.673741,0.731413,0.701393,0.636105,0.701595,0.667247
2,0.775955,0.711766,0.742476,0.750154,0.590834,0.661030,0.669023,0.639625,0.653994,0.649071,...,0.689116,0.695308,0.682452,0.688820,0.657999,0.705884,0.681101,0.677816,0.736740,0.706050
3,0.695167,0.724142,0.709359,0.659730,0.614271,0.636190,0.674246,0.654754,0.664357,0.634675,...,0.697208,0.665577,0.761002,0.710098,0.653899,0.781621,0.712078,0.668633,0.727307,0.696737
4,0.681680,0.608897,0.643236,0.651160,0.601042,0.625098,0.618116,0.594370,0.606011,0.637715,...,0.657648,0.632436,0.662249,0.646999,0.659560,0.697301,0.677906,0.650176,0.691115,0.670020


In [11]:
new_summary_dataset = pd.concat([summary_dataset, new_data_df.set_index(	mbart_predictions_bleu	mt5_predictions_bleu	summarunner_predictions_bleu	llama_7b_predictions_bleu	starling_predictions_bleu	yagpt_predictions_bleu	yagpt3_predictions_bleu
66	0.03623123312519232	0.012091077054298396	0.061175900934513854	0.03638503246078792	0.042085367700476925	0.023872420150931817	0.019104499120758397
67	0.08244459058186077	0.0070612410290086984	0.04337983821381413	0.03255433540584784	0.05226225627379292	0.08356109137266851	0.02447167872800763
68	0.1339050680154623	0.015023241957092445	0.08896750341231908	0.025705801685080742	0.06262411456218506	0.037450601740282585	0.05608640868147403
69	0.0594017179198919	0.034029823463094445	0.07232031895361328	0.023836806085825016	0.043464817648593876	0.028014127396018124	0.061944612569869696
70	0.03127720688173377	0.023399580998690737	0.02811915637645869	0.0412593678548575	0.040175964641610475	0.03871384728881924	0.04279592866572136
71	0.0935172384340886	0.00018733031312566438	0.08578556487203123	0.09546011651659139	0.11053328708170311	0.04107830331918027	0.12903758633524678
72	0.10103546726214899	0.09918081745590007	0.06252619258547598	0.09241363807310995	0.0879578773160082	0.03366125646849639	0.027968360509611868
73	0.043081749890020074	0.006032651928831756	0.06713879245223353	0.04714894753552036	0.10548568487707735	0.05471361361166315	0.06793375012047573
74	0.2170090032371664	0.009641738840568959	0.23491345700456148	0.07233886773490478	0.27736257718118196	0.11603274742444249	0.17060928518931617
75	0.05437119554989698	6.095972613178098e-05	0.05487086480387281	0.0196165597037266	0.03859341792359365	0.025325733635460056	0.02120207527968304
76	0.16040405825254334	0.023325605013096676	0.12492077997240149	0.03934751334194618	0.09999478328911843	0.05789960385094546	0.06865522713382077
77	0.05170470336523694	0.013780414588636172	0.05191583931604435	0.02374160158917232	0.04996128828133616	0.060837591402790214	0.08097559540055488
78	0.06312871110271528	0.10610335259861256	0.04532924433704278	0.028075444935609498	0.05073412597885699	0.054546701295990443	0.0699363486683451
79	0.1035333482068235	0.07052640671528307	0.05617590640776226	0.021295402627473457	0.03463978897835301	0.06407087505154628	0.06882160869839149
80	0.34371114431899924	0.02481080293018641	0.2784131667304188	0.0815992922611855	0.0559691785824413	0.0900980600740037	0.12885119253120664
81	0.1414630052412016	0.0020678647562976295	0.03760859546168579	0.030307162020993306	0.030764093861732473	0.04350699877338095	0.021818903131420007
82	0.055429001796335337	0.015415741703706255	0.026067261737789418	0.05366980771608109	0.07602327430712104	0.03418872796322438	0.022829847423290653
83	0.04817131454791843	0.01584770786147954	0.03364699525779844	0.029033045699873503	0.01794930043746585	0.04223604573063158	0.04269234003902816
84	0.09969434990349836	0.0757953917293172	0.04873146370092589	0.02652198622802939	0.042166709308030896	0.05410957969561705	0.045157140680870546
85	0.04629588336164947	0.05670171888988679	0.04428905434202579	0.01828273980995128	0.037340129118938706	0.03476701647658136	0.03873582507075529
86	0.03202289411234805	0.023249971609442588	0.030412544082680727	0.015132466299079158	0.020615239513638025	0.01584827417401328	0.01777724594642383
87	0.048525724220646695	0.02655579990415782	0.07553244389947197	0.025586840481909224	0.025652584894049743	0.01782412922849	0.02874368217704366
88	0.08150373937711172	0.026125764319396377	0.03897850283857833	0.0628310914955508	0.04162122586655054	0.04067335445752808	0.07391908926173968
89	0.036065029030476495	0.013318324522176288	0.032005121753835464	0.031380634748433014	0.025473887996172746	0.06957350729943415	0.03435763505835022
90	0.06558144622598075	0.03471384121503611	0.06625722873170614	0.02149647751327627	0.05691404903063441	0.018117937892902128	0.07666911020597449
91	0.052235777407450855	0.029628660049288276	0.038339132226749034	0.03029202698143069	0.02690670572409657	0.030912803732866524	0.056093243250242925
92	0.09210985163597842	0.12408865471638249	0.05114322273059111	0.04787030098785579	0.046163829144820624	0.04200594542496673	0.05964418182268273
93	0.08687475782716618	0.013868750553259871	0.10372336028581605	0.04214426245747872	0.02864049170676041	0.05801776522410952	0.04425184981184798
94	0.030471243850648387	0.01027404507375745	0.02519018440024851	0.022817750826207284	0.03576373874206134	0.034202077959679056	0.023570851710527026
95	0.06320785226224562	0.008039355526288495	0.039404726963488165	0.01700569790719272	0.03785661661441521	0.03223024979946278	0.03144406647233181
96	0.06944235967151513	0.024436624496676326	0.041927070297698404	0.0864701334218824	0.04446591151737814	0.039493818926492695	0.027620227857333722
97	0.1754475584010989	0.04013226316058926	0.09092617426809149	0.05915367110165543	0.037820544623490826	0.02159418781774839	0.022005485861068718
98	0.043156216058044795	0.05746428534616737	0.060529150006732486	0.017403248742231186	0.010186025882149478	0.019818753009172085	0.027903786634678848
99	0.035126318028803	0.000868791939481722	0.023148451361271284	0.029056669244838816	0.06094248770129678	0.025510426883257536	0.0811563673673393
100	0.06232809287848436	0.007216117323685777	0.03659068094150952	0.055009718989318765	0.03895317630669626	0.035899835697611404	0.04811212639299925
101	0.08802880726292506	0.059060243380765765	0.07850083484043821	0.04585331298730776	0.0984683104204656	0.035087154129185075	0.027471318708441127
102	0.13125283486625716	0.0065331437882746625	0.45808625270371234	0.11333224314953227	0.22038640390240094	0.10192986282105125	0.11090689733307425
103	0.08181946133557842	0.020973764711593613	0.18826300978111685	0.053199468920053415	0.05886662900746659	0.026244139974367954	0.06725110619469749
104	0.02310752826832743	1.6774442330736117e-06	0.0220852370973088	0.01603716833916845	0.043086791916568994	0.02912797895135938	0.03006347925387719
105	0.3025452919358529	0.015997694435376523	0.045531249598195975	0.12235485380532396	0.06818438896004227	0.045695100124667425	0.0959471190220409
106	0.1259323668397466	0.1103541462115306	0.03342309209236609	0.03948461770816673	0.11310321982396894	0.056337864726924286	0.10449095130554766
107	0.08863005960162665	0.008275283537560912	0.046772118579698624	0.028120506893037404	0.05358141605769965	0.035762174188747387	0.03574613978248222
108	0.0815825067025588	0.18511281569566135	0.08107879481375063	0.10405655451679935	0.06215837221963285	0.0971445094503852	0.02734195277682282
109	0.09657839701924877	0.0001085191465856552	0.10373032985214813	0.042347663624447605	0.04337856039152671	0.03314467417462596	0.03240113904384493
110	0.0399804080138911	0.0371587558902144	0.03953653303355168	0.0261489899398435	0.021344301601474696	0.018235514677841683	0.031069874854278538
111	0.032635848884021754	0.03921668938051627	0.03310295979899219	0.015371377335529868	0.02822752301534681	0.024033845490166322	0.028440172000217884
112	0.12640155906567838	0.03185715838489127	0.11431674406127963	0.03822496962987343	0.12950606233257028	0.03158483792418979	0.038524408605448764
113	0.35145869954457126	0.00013934134241324402	0.06393146921846518	0.05645129623781849	0.25927968197148304	0.07442687151977624	0.13267485717207844
114	0.08868947478646896	0.05271681819976374	0.040614262725559264	0.05188442445408188	0.042765647569886045	0.04293892258919701	0.0345005829248814
115	0.06981264763301163	0.007544949380953279	0.08863163543015472	0.043842394980138824	0.049051966966495324	0.03014812009788288	0.07578261312889117
116	0.06474997203797597	0.06718044979147227	0.044666237958045474	0.04117516541350685	0.03407251137531691	0.032722457038461196	0.038762940641239724
117	0.038607066990178344	0.0017759463471694135	0.032047574588556255	0.07111932424743476	0.11209655695156855	0.033910323294466395	0.03967783907658738
118	0.06550710485781203	0.026650333755309077	0.02800077436515644	0.03171028261088965	0.031627024515008414	0.04510251287361495	0.049888321711149916
119	0.06061744951206098	0.009964802786617457	0.03386431734183275	0.02575862680067149	0.03545783470016862	0.03185707716038172	0.03425487791780259
120	0.03282035983809527	0.03023043814245775	0.030661771012157113	0.01646318293539861	0.03667530253927969	0.016894573314992594	0.0153192471170326
121	0.1169019673217699	0.013738115910544647	0.09150844206032527	0.05433459629844592	0.052016901491022415	0.05387337077399883	0.09068109716346996
122	0.05710282127070634	0.03146160211379351	0.05410734354182638	0.02426281572042655	0.04578771908722105	0.03790191464705673	0.026382148372547373
123	0.0416362048454059	0.08934685264503363	0.045607095866301794	0.04979379803674504	0.02356042570031431	0.047908316773214076	0.06555829537445526
124	0.04796022802247918	0.004373220900833331	0.05194101910374519	0.08628466671759209	0.06805966213829301	0.032352989639388985	0.10516286723746036
125	0.025002550063466078	0.010079365732901172	0.02725484516447649	0.03485346282653492	0.051265852711587415	0.03912312183166831	0.034709562062931194
126	0.22502261458865166	0.00877833441791453	0.05861484411000976	0.06252455523157313	0.11071682869684163	0.08792627767590006	0.06863341491691023
127	0.060837606285167045	0.05244245648114473	0.20080041524158598	0.03816630824856837	0.057575739712018285	0.09872461311031235	0.0373623566672542
128	0.04184227397648825	0.010938988158657882	0.09470897027997424	0.029324407711832604	0.038553764509237566	0.07210845914340339	0.07531374295486135
129	0.2039823758593055	0.03471384121503611	0.1859366376012899	0.05317083352144114	0.163566248014279	0.0504938851299323	0.267342795498952.index)], axis=1)
new_summary_dataset.head()
print(len(new_summary_dataset))

65


In [12]:
summary_dataset = pd.DataFrame(new_summary_dataset)
del new_summary_dataset

In [13]:
summary_dataset.head()
print(len(summary_dataset))

65


In [14]:
from statistics import mean 
from collections import defaultdict
score_stats = defaultdict(dict) if (not "score_stats" in locals() and not "score_stats" in globals()) else score_stats
for filename in files:
    predictions_field = filename.split(".")[0]
    
    mean_f1 = mean(results[predictions_field][f"{predictions_field}_bertscore_f1"])
    var_f1 = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_bertscore_f1"])) - mean_f1 ** 2
    
    mean_precision = mean(results[predictions_field][f"{predictions_field}_bertscore_precision"])
    var_precision = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_bertscore_precision"])) - mean_precision ** 2
    
    mean_recall = mean(results[predictions_field][f"{predictions_field}_bertscore_recall"])
    var_recall = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_bertscore_recall"])) - mean_recall ** 2
    
    score_stats[predictions_field].update({"mean_precision": mean_precision, "var_precision": var_precision, "mean_recall": mean_recall, "var_recall": var_recall, "mean_f1": mean_f1, "var_f1": var_f1})
print(dict(score_stats))

{'mbart_predictions': {'mean_precision': 0.690050169137808, 'var_precision': 0.0019415854025952584, 'mean_recall': 0.6960230689782363, 'var_recall': 0.0025041194168496927, 'mean_f1': 0.6922733288544874, 'var_f1': 0.0017058744625975675}, 'mt5_predictions': {'mean_precision': 0.7098548549872178, 'var_precision': 0.0035838504630282264, 'mean_recall': 0.6099580462162312, 'var_recall': 0.002848614263516347, 'mean_f1': 0.6551174383897048, 'var_f1': 0.002486519195151027}, 'summarunner_predictions': {'mean_precision': 0.6632669403002812, 'var_precision': 0.002033478337890249, 'mean_recall': 0.676481529382559, 'var_recall': 0.002610037750397254, 'mean_f1': 0.669170290690202, 'var_f1': 0.0018536016713037173}, 'llama_7b_predictions': {'mean_precision': 0.6443997227228605, 'var_precision': 0.0008212485301306582, 'mean_recall': 0.7223757982254029, 'var_recall': 0.0018553037894400992, 'mean_f1': 0.6805312119997464, 'var_f1': 0.0008262457082601204}, 'starling_predictions': {'mean_precision': 0.665734

In [15]:
results.keys()

dict_keys(['mbart_predictions', 'mt5_predictions', 'summarunner_predictions', 'llama_7b_predictions', 'starling_predictions', 'yagpt_predictions', 'yagpt3_predictions'])

In [16]:
stats_pd = pd.DataFrame(score_stats)

In [17]:
stats_pd.head(None)

,mbart_predictions,mt5_predictions,summarunner_predictions,llama_7b_predictions,starling_predictions,yagpt_predictions,yagpt3_predictions
mean_precision,0.690050,0.709855,0.663267,0.644400,0.665735,0.657500,0.654002
var_precision,0.001942,0.003584,0.002033,0.000821,0.002084,0.000793,0.001399
mean_recall,0.696023,0.609958,0.676482,0.722376,0.706216,0.714787,0.723105
var_recall,0.002504,0.002849,0.002610,0.001855,0.001771,0.001589,0.004834
mean_f1,0.692273,0.655117,0.669170,0.680531,0.684643,0.684335,0.685927
var_f1,0.001706,0.002487,0.001854,0.000826,0.001501,0.000723,0.002699


In [18]:
summary_dataset.to_csv("summary_dataset_with_bertscore.csv")

## ROUGE

In [19]:
import pandas as pd
summary_dataset = pd.read_csv("summary_dataset_with_bertscore.csv")
files = [
    "mbart_predictions.txt",
    "mt5_predictions.txt",
    "summarunner_predictions.txt",
    "llama_7b_predictions.csv",
    "starling_predictions.csv",
    "yagpt_predictions.csv",
    "yagpt3_predictions.csv"
]

In [20]:
import razdel
from evaluate import load
# results = {}
rouge = load('rouge')
references = list(summary_dataset["summary"])

def a(x):
    tokens = list(razdel.tokenize(x))
    return [_.text for _ in tokens]
    # return x.split()

for filename in files:
    predictions_field = filename.split(".")[0]
    predictions = list(summary_dataset[predictions_field])
    # results[predictions_field] = rouge.compute(predictions=predictions, references=references, use_aggregator=False, tokenizer=lambda x : list(razdel.tokenize(x)))
    results[predictions_field] = rouge.compute(predictions=predictions, references=references, use_aggregator=False, tokenizer=a)

In [21]:
data_keys = list(results.keys())
for data_key in data_keys:
    old_keys = list(results[data_key].keys())
    for old_metric_key in old_keys:
        results[data_key][f"{data_key}_" + old_metric_key] = results[data_key].pop(old_metric_key)

In [22]:
print(results[filename.split(".")[0]].keys())

dict_keys(['yagpt3_predictions_rouge1', 'yagpt3_predictions_rouge2', 'yagpt3_predictions_rougeL', 'yagpt3_predictions_rougeLsum'])


In [23]:
print(len(results[filename.split(".")[0]][f"{filename.split('.')[0]}_rouge1"]))

65


In [24]:
new_data = {}
for data_key in results.keys():
    for metric_key in results[data_key].keys():
        if "hashcode" in metric_key:
            continue
        new_data[metric_key] = pd.Series(results[data_key][metric_key])
new_data_df = pd.DataFrame(new_data)
new_data_df.head()

,mbart_predictions_rouge1,mbart_predictions_rouge2,mbart_predictions_rougeL,mbart_predictions_rougeLsum,mt5_predictions_rouge1,mt5_predictions_rouge2,mt5_predictions_rougeL,mt5_predictions_rougeLsum,summarunner_predictions_rouge1,summarunner_predictions_rouge2,...,starling_predictions_rougeL,starling_predictions_rougeLsum,yagpt_predictions_rouge1,yagpt_predictions_rouge2,yagpt_predictions_rougeL,yagpt_predictions_rougeLsum,yagpt3_predictions_rouge1,yagpt3_predictions_rouge2,yagpt3_predictions_rougeL,yagpt3_predictions_rougeLsum
0,0.228070,0.017857,0.157895,0.157895,0.151515,0.000000,0.090909,0.090909,0.262626,0.041237,...,0.162162,0.162162,0.176101,0.025478,0.125786,0.138365,0.171429,0.011561,0.102857,0.114286
1,0.368932,0.118812,0.330097,0.330097,0.202532,0.025974,0.202532,0.202532,0.269841,0.032258,...,0.256410,0.256410,0.310160,0.086486,0.235294,0.171123,0.257732,0.031250,0.164948,0.175258
2,0.395604,0.179775,0.351648,0.351648,0.176471,0.090909,0.147059,0.147059,0.372549,0.060000,...,0.214286,0.214286,0.241546,0.058537,0.164251,0.164251,0.309677,0.078431,0.193548,0.245161
3,0.333333,0.063830,0.208333,0.208333,0.169492,0.000000,0.169492,0.169492,0.233766,0.053333,...,0.174497,0.174497,0.239130,0.054945,0.173913,0.173913,0.317757,0.095238,0.224299,0.205607
4,0.175824,0.022472,0.131868,0.131868,0.195122,0.025000,0.170732,0.170732,0.142857,0.000000,...,0.198473,0.198473,0.321678,0.056738,0.223776,0.223776,0.285714,0.108434,0.190476,0.238095


In [25]:
new_summary_dataset = pd.concat([summary_dataset, new_data_df], axis=1)
new_summary_dataset.head()

,Unnamed: 0,Unnamed: 0.1,Ind,title,text,summary,mbart_predictions,mt5_predictions,summarunner_predictions,llama_7b_predictions,...,starling_predictions_rougeL,starling_predictions_rougeLsum,yagpt_predictions_rouge1,yagpt_predictions_rouge2,yagpt_predictions_rougeL,yagpt_predictions_rougeLsum,yagpt3_predictions_rouge1,yagpt3_predictions_rouge2,yagpt3_predictions_rougeL,yagpt3_predictions_rougeLsum
0,53,53,53,Названа опасность постоянно включенного Blueto...,Постоянно включенный Bluetooth на смартфоне гр...,Активированный в смартфоне Bluetooth может пре...,Постоянно включенный Bluetooth на смартфоне гр...,"Эксперты предупреждают о том, что отключать Bl...",постоянно включенный bluetooth на смартфоне гр...,"Bluetooth постоянно включен, это опасно, так к...",...,0.162162,0.162162,0.176101,0.025478,0.125786,0.138365,0.171429,0.011561,0.102857,0.114286
1,244,244,244,Колесникова проходит подозреваемой по делу о з...,Член президиума координационного совета оппози...,Члена президиума оппозиционного Координационно...,Член президиума координационного совета оппози...,Член президиума координационного совета оппози...,член президиума координационного совета оппози...,"Мария Колесникова, член президиума координацио...",...,0.256410,0.256410,0.310160,0.086486,0.235294,0.171123,0.257732,0.031250,0.164948,0.175258
2,260,260,260,Deutsche Bank: в мире наступает эпоха беспорядка,Аналитики Deutsche Bank обнародовали исследова...,В истории человечества наступает эпоха беспоря...,В 2020 году в мире наступит новая эпоха беспор...,Пандемия коронавируса привела к появлениям нов...,аналитики deutsche bank обнародовали исследова...,Аналитики Deutsche Bank предсказали наступлени...,...,0.214286,0.214286,0.241546,0.058537,0.164251,0.164251,0.309677,0.078431,0.193548,0.245161
3,355,355,355,«Ъ»: Минтранс подготовил проект поправок к ПДД,ГИБДД совместно с Минтрансом разработала масшт...,Масштабный проект изменений в ПДД подготовили ...,ГИБДД совместно с Минтрансом разработала масшт...,В Госдуму вступили в силу поправки в правила д...,гибдд совместно с минтрансом разработала масшт...,ГИБДД и Минтранс разработали проект изменений ...,...,0.174497,0.174497,0.239130,0.054945,0.173913,0.173913,0.317757,0.095238,0.224299,0.205607
4,653,653,653,Tesla выпустит бюджетный беспилотник в 2023 году,Вечером 22 сентября на конференции Battery Day...,Миллиардер Илон Маск пообещал представить бюдж...,На конференции Battery Day глава Tesla Илон Ма...,Илон Маск рассчитывает вывести на рынок бюджет...,вечером 22 сентября на конференции battery day...,Илон Маск представил планы Tesla по выходу на ...,...,0.198473,0.198473,0.321678,0.056738,0.223776,0.223776,0.285714,0.108434,0.190476,0.238095


In [26]:
summary_dataset = pd.DataFrame(new_summary_dataset)
del new_summary_dataset

In [27]:
summary_dataset.head()

,Unnamed: 0,Unnamed: 0.1,Ind,title,text,summary,mbart_predictions,mt5_predictions,summarunner_predictions,llama_7b_predictions,...,starling_predictions_rougeL,starling_predictions_rougeLsum,yagpt_predictions_rouge1,yagpt_predictions_rouge2,yagpt_predictions_rougeL,yagpt_predictions_rougeLsum,yagpt3_predictions_rouge1,yagpt3_predictions_rouge2,yagpt3_predictions_rougeL,yagpt3_predictions_rougeLsum
0,53,53,53,Названа опасность постоянно включенного Blueto...,Постоянно включенный Bluetooth на смартфоне гр...,Активированный в смартфоне Bluetooth может пре...,Постоянно включенный Bluetooth на смартфоне гр...,"Эксперты предупреждают о том, что отключать Bl...",постоянно включенный bluetooth на смартфоне гр...,"Bluetooth постоянно включен, это опасно, так к...",...,0.162162,0.162162,0.176101,0.025478,0.125786,0.138365,0.171429,0.011561,0.102857,0.114286
1,244,244,244,Колесникова проходит подозреваемой по делу о з...,Член президиума координационного совета оппози...,Члена президиума оппозиционного Координационно...,Член президиума координационного совета оппози...,Член президиума координационного совета оппози...,член президиума координационного совета оппози...,"Мария Колесникова, член президиума координацио...",...,0.256410,0.256410,0.310160,0.086486,0.235294,0.171123,0.257732,0.031250,0.164948,0.175258
2,260,260,260,Deutsche Bank: в мире наступает эпоха беспорядка,Аналитики Deutsche Bank обнародовали исследова...,В истории человечества наступает эпоха беспоря...,В 2020 году в мире наступит новая эпоха беспор...,Пандемия коронавируса привела к появлениям нов...,аналитики deutsche bank обнародовали исследова...,Аналитики Deutsche Bank предсказали наступлени...,...,0.214286,0.214286,0.241546,0.058537,0.164251,0.164251,0.309677,0.078431,0.193548,0.245161
3,355,355,355,«Ъ»: Минтранс подготовил проект поправок к ПДД,ГИБДД совместно с Минтрансом разработала масшт...,Масштабный проект изменений в ПДД подготовили ...,ГИБДД совместно с Минтрансом разработала масшт...,В Госдуму вступили в силу поправки в правила д...,гибдд совместно с минтрансом разработала масшт...,ГИБДД и Минтранс разработали проект изменений ...,...,0.174497,0.174497,0.239130,0.054945,0.173913,0.173913,0.317757,0.095238,0.224299,0.205607
4,653,653,653,Tesla выпустит бюджетный беспилотник в 2023 году,Вечером 22 сентября на конференции Battery Day...,Миллиардер Илон Маск пообещал представить бюдж...,На конференции Battery Day глава Tesla Илон Ма...,Илон Маск рассчитывает вывести на рынок бюджет...,вечером 22 сентября на конференции battery day...,Илон Маск представил планы Tesla по выходу на ...,...,0.198473,0.198473,0.321678,0.056738,0.223776,0.223776,0.285714,0.108434,0.190476,0.238095


Посчитаем статистики для представления о качестве датасета

In [28]:
from statistics import mean 
from collections import defaultdict
score_stats = defaultdict(dict) if (not "score_stats" in locals() and not "score_stats" in globals()) else score_stats
for filename in files:
    predictions_field = filename.split(".")[0]
    mean_rouge1 = mean(results[predictions_field][f"{predictions_field}_rouge1"])
    var_rouge1 = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_rouge1"])) - mean_rouge1 ** 2
    
    mean_rouge2 = mean(results[predictions_field][f"{predictions_field}_rouge2"])
    var_rouge2 = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_rouge2"])) - mean_rouge2 ** 2
    
    mean_rougeL = mean(results[predictions_field][f"{predictions_field}_rougeL"])
    var_rougeL = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_rougeL"])) - mean_rougeL ** 2
    
    mean_rougeLsum = mean(results[predictions_field][f"{predictions_field}_rougeLsum"])
    var_rougeLsum = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_rougeLsum"])) - mean_rougeLsum ** 2
    score_stats[predictions_field].update({"mean_rouge1": mean_rouge1, "var_rouge1": var_rouge1, "mean_rouge2": mean_rouge2, "var_rouge2": var_rouge2, "mean_rougeL": mean_rougeL, "var_rougeL": var_rougeL, "mean_rougeLsum": mean_rougeLsum, "var_rougeLsum": var_rougeLsum})
print(dict(score_stats))

{'mbart_predictions': {'mean_precision': 0.690050169137808, 'var_precision': 0.0019415854025952584, 'mean_recall': 0.6960230689782363, 'var_recall': 0.0025041194168496927, 'mean_f1': 0.6922733288544874, 'var_f1': 0.0017058744625975675, 'mean_rouge1': 0.3132584410306658, 'var_rouge1': 0.008066642067746585, 'mean_rouge2': 0.10460121341018001, 'var_rouge2': 0.007166239563576184, 'mean_rougeL': 0.22920292756039287, 'var_rougeL': 0.007787845278402608, 'mean_rougeLsum': 0.22920292756039287, 'var_rougeLsum': 0.007787845278402608}, 'mt5_predictions': {'mean_precision': 0.7098548549872178, 'var_precision': 0.0035838504630282264, 'mean_recall': 0.6099580462162312, 'var_recall': 0.002848614263516347, 'mean_f1': 0.6551174383897048, 'var_f1': 0.002486519195151027, 'mean_rouge1': 0.18770184990638486, 'var_rouge1': 0.008669013764652747, 'mean_rouge2': 0.055957208707190335, 'var_rouge2': 0.004357179302824177, 'mean_rougeL': 0.15503712316282722, 'var_rougeL': 0.0064435302208476555, 'mean_rougeLsum': 0.

In [29]:
stats_pd = pd.DataFrame(score_stats)

In [30]:
stats_pd.head(len(stats_pd))

,mbart_predictions,mt5_predictions,summarunner_predictions,llama_7b_predictions,starling_predictions,yagpt_predictions,yagpt3_predictions
mean_precision,0.690050,0.709855,0.663267,0.644400,0.665735,0.657500,0.654002
var_precision,0.001942,0.003584,0.002033,0.000821,0.002084,0.000793,0.001399
mean_recall,0.696023,0.609958,0.676482,0.722376,0.706216,0.714787,0.723105
var_recall,0.002504,0.002849,0.002610,0.001855,0.001771,0.001589,0.004834
mean_f1,0.692273,0.655117,0.669170,0.680531,0.684643,0.684335,0.685927
var_f1,0.001706,0.002487,0.001854,0.000826,0.001501,0.000723,0.002699
mean_rouge1,0.313258,0.187702,0.277396,0.236332,0.279708,0.237622,0.263621
var_rouge1,0.008067,0.008669,0.009279,0.003580,0.005227,0.003213,0.004378
mean_rouge2,0.104601,0.055957,0.073982,0.067198,0.078820,0.069057,0.082772
var_rouge2,0.007166,0.004357,0.007102,0.001667,0.003717,0.001433,0.003424


In [31]:
summary_dataset.to_csv("summary_dataset_with_bertscore_rouge.csv")

## METEOR

https://huggingface.co/spaces/evaluate-metric/meteor

In [32]:
import pandas as pd
summary_dataset = pd.read_csv("summary_dataset_with_bertscore_rouge.csv")
files = [
    "mbart_predictions.txt",
    "mt5_predictions.txt",
    "summarunner_predictions.txt",
    "llama_7b_predictions.csv",
    "starling_predictions.csv",
    "yagpt_predictions.csv",
    "yagpt3_predictions.csv"
]

In [33]:
summary_dataset.columns

Index(['Unnamed: 0.2', 'Unnamed: 0', 'Unnamed: 0.1', 'Ind', 'title', 'text',
       'summary', 'mbart_predictions', 'mt5_predictions',
       'summarunner_predictions', 'llama_7b_predictions',
       'starling_predictions', 'yagpt_predictions', 'yagpt3_predictions',
       'mbart_predictions_bertscore_precision',
       'mbart_predictions_bertscore_recall', 'mbart_predictions_bertscore_f1',
       'mt5_predictions_bertscore_precision',
       'mt5_predictions_bertscore_recall', 'mt5_predictions_bertscore_f1',
       'summarunner_predictions_bertscore_precision',
       'summarunner_predictions_bertscore_recall',
       'summarunner_predictions_bertscore_f1',
       'llama_7b_predictions_bertscore_precision',
       'llama_7b_predictions_bertscore_recall',
       'llama_7b_predictions_bertscore_f1',
       'starling_predictions_bertscore_precision',
       'starling_predictions_bertscore_recall',
       'starling_predictions_bertscore_f1',
       'yagpt_predictions_bertscore_precision',

In [34]:
from tqdm import tqdm
from threading import Thread
from evaluate import load
import os, contextlib

import nltk
nltk.download('wordnet')


def calculate_meteor(result_list, predictions, references, thread_no, step):
    meteor = None
    # with open(os.devnull, 'w') as devnull:
    #     with contextlib.redirect_stdout(devnull), contextlib.redirect_stderr(devnull):
    
    meteor = load('meteor')
    index = thread_no
    disable_tqdm = (thread_no != 0)
    
    
    with tqdm(total=len(predictions), disable=disable_tqdm) as pbar:
        pbar.update(index)
        while index < len(predictions):
            result_list[index] = meteor.compute(predictions=[predictions[index]], references=[references[index]])
            index += step
            pbar.update(step)
    print(f"Thread {thread_no} finished")

results = {}
references = list(summary_dataset["summary"])
for filename in files:
    predictions_field = filename.split(".")[0]
    predictions = list(summary_dataset[predictions_field])
    results[predictions_field] = [0] * len(predictions)
    
    thread_num = 32
    thread_pool = []
    for thread in range(thread_num):
        thread_pool.append(Thread(target=calculate_meteor, args=[results[predictions_field], predictions, references, thread, thread_num], daemon=True))
        thread_pool[-1].start()

    for thread in range(thread_num):
        thread_pool[thread].join()

[nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/skatori/nltk_data...
[nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/skatori/nltk_data...
[nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/skatori/nltk_data...
[nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/skatori/nltk_data...
[nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up

Thread 4 finished
Thread 18 finished
Thread 28 finished
Thread 13 finished
Thread 3 finished
Thread 6 finished
Thread 8 finished


[nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/skatori/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/skatori/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
96it [00:01, 86.13it/s]                                                         
[nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/skatori/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/skatori/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Thread 0 finished
Thread 2 finished
Thread 23 finished
Thread 5 finished
Thread 16 finished
Thread 7 finished
Thread 24 finished
Thread 19 finished
Thread 10 finished
Thread 14 finished
Thread 22 finished
Thread 31 finished
Thread 12 finished
Thread 30 finished
Thread 25 finished
Thread 11 finished
Thread 27 finished
Thread 21 finished
Thread 1 finished
Thread 20 finished
Thread 26 finished
Thread 29 finished
Thread 9 finished
Thread 17 finished
Thread 15 finished


[nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/skatori/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/skatori/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/skatori/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/skatori/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/skatori/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading pack

...

Thread 31 finished
Thread 1 finished
Thread 4 finished
Thread 13 finished
Thread 23 finished
Thread 22 finished
Thread 8 finished
Thread 2 finished
Thread 20 finished


[nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/skatori/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/skatori/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/skatori/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/skatori/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/skatori/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading pack

Thread 9 finished


[nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/skatori/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/skatori/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/skatori/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/skatori/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/skatori/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading pack

Thread 26 finished
Thread 3 finished
Thread 12 finished
Thread 7 finished
Thread 27 finished
Thread 17 finished
Thread 21 finished
Thread 6 finished
Thread 11 finished
Thread 25 finished
Thread 5 finished
Thread 15 finished


In [35]:
print(results)

{'mbart_predictions': [{'meteor': 0.1264591439688716}, {'meteor': 0.24882943143812708}, {'meteor': 0.30963082823924243}, {'meteor': 0.3063148788927336}, {'meteor': 0.09889056691449813}, {'meteor': 0.21086415830573935}, {'meteor': 0.16566965589606567}, {'meteor': 0.17318553669684708}, {'meteor': 0.4043802922945224}, {'meteor': 0.2147186147186147}, {'meteor': 0.4260436681222708}, {'meteor': 0.27991741443659174}, {'meteor': 0.27381806227960076}, {'meteor': 0.254421768707483}, {'meteor': 0.5005431727775856}, {'meteor': 0.33779877260981916}, {'meteor': 0.13901212658976636}, {'meteor': 0.18330585509312894}, {'meteor': 0.26894287224865737}, {'meteor': 0.2222222222222222}, {'meteor': 0.14947216465628707}, {'meteor': 0.23696177609297744}, {'meteor': 0.28956422018348627}, {'meteor': 0.08445945945945944}, {'meteor': 0.21568906605922553}, {'meteor': 0.13054830287206265}, {'meteor': 0.26161634244341014}, {'meteor': 0.21879634206623827}, {'meteor': 0.1358234295415959}, {'meteor': 0.1655676959619953}

In [36]:
for key in results.keys():
    results[key] = {f"{key}_meteor": [val["meteor"] if not isinstance(val, int) else 0 for val in results[key]]}
    print(key, len(results[key]))

mbart_predictions 1
mt5_predictions 1
summarunner_predictions 1
llama_7b_predictions 1
starling_predictions 1
yagpt_predictions 1
yagpt3_predictions 1


In [37]:
new_data = {}
for data_key in results.keys():
    for metric_key in results[data_key].keys():
        if "hashcode" in metric_key:
            continue
        new_data[metric_key] = pd.Series(results[data_key][metric_key])
new_data_df = pd.DataFrame(new_data)
new_data_df.head()

,mbart_predictions_meteor,mt5_predictions_meteor,summarunner_predictions_meteor,llama_7b_predictions_meteor,starling_predictions_meteor,yagpt_predictions_meteor,yagpt3_predictions_meteor
0,0.126459,0.053648,0.168245,0.207257,0.127202,0.168304,0.139373
1,0.248829,0.101734,0.263807,0.231588,0.310913,0.327753,0.224195
2,0.309631,0.086664,0.314012,0.262029,0.238650,0.295642,0.311487
3,0.306315,0.082645,0.233184,0.256739,0.270406,0.338463,0.296722
4,0.098891,0.100573,0.146562,0.227205,0.193430,0.202198,0.251442


In [38]:
new_summary_dataset = pd.concat([summary_dataset, new_data_df], axis=1)
new_summary_dataset.head()

,Unnamed: 0.2,Unnamed: 0,Unnamed: 0.1,Ind,title,text,summary,mbart_predictions,mt5_predictions,summarunner_predictions,...,yagpt3_predictions_rouge2,yagpt3_predictions_rougeL,yagpt3_predictions_rougeLsum,mbart_predictions_meteor,mt5_predictions_meteor,summarunner_predictions_meteor,llama_7b_predictions_meteor,starling_predictions_meteor,yagpt_predictions_meteor,yagpt3_predictions_meteor
0,0,53,53,53,Названа опасность постоянно включенного Blueto...,Постоянно включенный Bluetooth на смартфоне гр...,Активированный в смартфоне Bluetooth может пре...,Постоянно включенный Bluetooth на смартфоне гр...,"Эксперты предупреждают о том, что отключать Bl...",постоянно включенный bluetooth на смартфоне гр...,...,0.011561,0.102857,0.114286,0.126459,0.053648,0.168245,0.207257,0.127202,0.168304,0.139373
1,1,244,244,244,Колесникова проходит подозреваемой по делу о з...,Член президиума координационного совета оппози...,Члена президиума оппозиционного Координационно...,Член президиума координационного совета оппози...,Член президиума координационного совета оппози...,член президиума координационного совета оппози...,...,0.031250,0.164948,0.175258,0.248829,0.101734,0.263807,0.231588,0.310913,0.327753,0.224195
2,2,260,260,260,Deutsche Bank: в мире наступает эпоха беспорядка,Аналитики Deutsche Bank обнародовали исследова...,В истории человечества наступает эпоха беспоря...,В 2020 году в мире наступит новая эпоха беспор...,Пандемия коронавируса привела к появлениям нов...,аналитики deutsche bank обнародовали исследова...,...,0.078431,0.193548,0.245161,0.309631,0.086664,0.314012,0.262029,0.238650,0.295642,0.311487
3,3,355,355,355,«Ъ»: Минтранс подготовил проект поправок к ПДД,ГИБДД совместно с Минтрансом разработала масшт...,Масштабный проект изменений в ПДД подготовили ...,ГИБДД совместно с Минтрансом разработала масшт...,В Госдуму вступили в силу поправки в правила д...,гибдд совместно с минтрансом разработала масшт...,...,0.095238,0.224299,0.205607,0.306315,0.082645,0.233184,0.256739,0.270406,0.338463,0.296722
4,4,653,653,653,Tesla выпустит бюджетный беспилотник в 2023 году,Вечером 22 сентября на конференции Battery Day...,Миллиардер Илон Маск пообещал представить бюдж...,На конференции Battery Day глава Tesla Илон Ма...,Илон Маск рассчитывает вывести на рынок бюджет...,вечером 22 сентября на конференции battery day...,...,0.108434,0.190476,0.238095,0.098891,0.100573,0.146562,0.227205,0.193430,0.202198,0.251442


In [39]:
summary_dataset = pd.DataFrame(new_summary_dataset)
del new_summary_dataset

In [40]:
summary_dataset.head()

,Unnamed: 0.2,Unnamed: 0,Unnamed: 0.1,Ind,title,text,summary,mbart_predictions,mt5_predictions,summarunner_predictions,...,yagpt3_predictions_rouge2,yagpt3_predictions_rougeL,yagpt3_predictions_rougeLsum,mbart_predictions_meteor,mt5_predictions_meteor,summarunner_predictions_meteor,llama_7b_predictions_meteor,starling_predictions_meteor,yagpt_predictions_meteor,yagpt3_predictions_meteor
0,0,53,53,53,Названа опасность постоянно включенного Blueto...,Постоянно включенный Bluetooth на смартфоне гр...,Активированный в смартфоне Bluetooth может пре...,Постоянно включенный Bluetooth на смартфоне гр...,"Эксперты предупреждают о том, что отключать Bl...",постоянно включенный bluetooth на смартфоне гр...,...,0.011561,0.102857,0.114286,0.126459,0.053648,0.168245,0.207257,0.127202,0.168304,0.139373
1,1,244,244,244,Колесникова проходит подозреваемой по делу о з...,Член президиума координационного совета оппози...,Члена президиума оппозиционного Координационно...,Член президиума координационного совета оппози...,Член президиума координационного совета оппози...,член президиума координационного совета оппози...,...,0.031250,0.164948,0.175258,0.248829,0.101734,0.263807,0.231588,0.310913,0.327753,0.224195
2,2,260,260,260,Deutsche Bank: в мире наступает эпоха беспорядка,Аналитики Deutsche Bank обнародовали исследова...,В истории человечества наступает эпоха беспоря...,В 2020 году в мире наступит новая эпоха беспор...,Пандемия коронавируса привела к появлениям нов...,аналитики deutsche bank обнародовали исследова...,...,0.078431,0.193548,0.245161,0.309631,0.086664,0.314012,0.262029,0.238650,0.295642,0.311487
3,3,355,355,355,«Ъ»: Минтранс подготовил проект поправок к ПДД,ГИБДД совместно с Минтрансом разработала масшт...,Масштабный проект изменений в ПДД подготовили ...,ГИБДД совместно с Минтрансом разработала масшт...,В Госдуму вступили в силу поправки в правила д...,гибдд совместно с минтрансом разработала масшт...,...,0.095238,0.224299,0.205607,0.306315,0.082645,0.233184,0.256739,0.270406,0.338463,0.296722
4,4,653,653,653,Tesla выпустит бюджетный беспилотник в 2023 году,Вечером 22 сентября на конференции Battery Day...,Миллиардер Илон Маск пообещал представить бюдж...,На конференции Battery Day глава Tesla Илон Ма...,Илон Маск рассчитывает вывести на рынок бюджет...,вечером 22 сентября на конференции battery day...,...,0.108434,0.190476,0.238095,0.098891,0.100573,0.146562,0.227205,0.193430,0.202198,0.251442


Посчитаем статистики для представления о качестве датасета

In [41]:
from collections import defaultdict

score_stats = defaultdict(dict) if (not "score_stats" in locals() and not "score_stats" in globals()) else score_stats
for filename in files:
    predictions_field = filename.split(".")[0]
    score_stats[predictions_field].update({f"meteor": sum(results[predictions_field][f"{predictions_field}_meteor"]) / len(results[predictions_field][f"{predictions_field}_meteor"])})

In [42]:
stats_pd = pd.DataFrame(score_stats)

In [43]:
stats_pd.head(len(stats_pd))

,mbart_predictions,mt5_predictions,summarunner_predictions,llama_7b_predictions,starling_predictions,yagpt_predictions,yagpt3_predictions
mean_precision,0.690050,0.709855,0.663267,0.644400,0.665735,0.657500,0.654002
var_precision,0.001942,0.003584,0.002033,0.000821,0.002084,0.000793,0.001399
mean_recall,0.696023,0.609958,0.676482,0.722376,0.706216,0.714787,0.723105
var_recall,0.002504,0.002849,0.002610,0.001855,0.001771,0.001589,0.004834
mean_f1,0.692273,0.655117,0.669170,0.680531,0.684643,0.684335,0.685927
var_f1,0.001706,0.002487,0.001854,0.000826,0.001501,0.000723,0.002699
mean_rouge1,0.313258,0.187702,0.277396,0.236332,0.279708,0.237622,0.263621
var_rouge1,0.008067,0.008669,0.009279,0.003580,0.005227,0.003213,0.004378
mean_rouge2,0.104601,0.055957,0.073982,0.067198,0.078820,0.069057,0.082772
var_rouge2,0.007166,0.004357,0.007102,0.001667,0.003717,0.001433,0.003424


In [44]:
summary_dataset.to_csv("summary_dataset_with_bertscore_rouge_meteor.csv")